In [1]:
from xml.etree import ElementTree as ET
import os
import csv
import pandas as pd

import numpy as np 
import matplotlib.pyplot as plt
import seaborn as sns
import nltk
from nltk import word_tokenize
from nltk.corpus import stopwords

In [4]:
class csvCreator:
    def xmlRaporuAl(self,report_name):
        tree = ET.parse(report_name)
        root = tree.getroot()
        return root

      # Id sayısı kadar rapor olusturacağım için önce kaç Id olduğunu buluyorum.
    def id_Bul(self,root):
        '''
        Bu fonksiyon raporların ilişkili olduğu görüntünün id sini bulmamıza yardımcı olur.
        '''
        my_Images = [] # Goruntuleri tutacak olan liste
        for img in root.getchildren(): # xml dosyası içinde ve root altındaki tüm tag'leri dolaş
            if img.tag == 'parentImage': # bu tagler'den parentImage eşit ise 
                img_id = list(img.attrib.values())[0] #attribute degerlerinin ilk elemanı bize raporun ait oldugu image id donduruyor
                my_Images.append(img_id) # bunu id'ler icin olusturdugum listeye ekle
        return my_Images 

    def finding_Bul(self,root):
        '''
        Bu fonksiyon xml tipindeki raporlar içerisindeki findings kısmını bulmamıza yardımcı olur.
        '''
        findings = root.find('./MedlineCitation/Article/Abstract/AbstractText[@Label="FINDINGS"]')

        finding_text = findings.text
        if finding_text == None:
            finding_text = 'No Findings'
        return finding_text


    def satirEkle(self,filename,report_id,finding):
        '''
        Bu fonksiyon bize sadece id ve findinglerin oldugu csv dosyasını oluşturmamızda yardımcı olur.
        inputs : report_id - raporumuzun ilişkili olduğu resmin id degeri
                finding - raporlardaki bulgumuz yani asıl raporu olusturan kısımdır.
        '''
        rows = [] # Değerlerimizi tutacak olan satır.

        report_as_csv = open(filename,'a',encoding='utf-8')

        #Değerler satırını oluştur ve csv içerisine yükle.
        rows.append(report_id)
        rows.append(finding)
        csv.writer(report_as_csv).writerow(rows)

        #close openin file .
        report_as_csv.close()

    def datasetiHazirla(self,kayit_dosyasi,goruntulerin_dosya_yolu):
        with open(kayit_dosyasi,'w',encoding='utf-8') as r: # Kayıt edilecek boş rapor csv dosyasını oluştur.
            columns = ['Id','Findings']
            csv.writer(r).writerow(columns)

        all_files_names = os.listdir(goruntulerin_dosya_yolu)    


        # Başlangıç..

        for file_name in all_files_names:
            if file_name.endswith('.xml'):    
                root = self.xmlRaporuAl(goruntulerin_dosya_yolu+file_name)
                print(file_name+" is loaded.\n")
            else:
                continue
            try:  
                my_images_all_id = cc.id_Bul(root)
                finding = self.finding_Bul(root)
                print("Report Id/s:",my_images_all_id,"\nFindings:",finding,"\n")
            except:
                print("Exception : Report informations can't get  !!")
            try:
                for id in my_images_all_id:
                    self.satirEkle(kayit_dosyasi,id,finding)
                    print(id," is appended successfully.\n\n*********\n")
                if my_images_all_id <1:
                    print("There is NO IMAGE data with related ",file_name)
            except:
                print("Exception : Values can't add to new report file.")

In [ ]:
kayit_dosyasi = 'metadata_all.csv'
goruntulerin_dosya_yolu = './dataset/reports/'
cc = csvCreator() # CSV dosyasını dolduracak olan işlemleri gerçekleştiren objeyi oluştur.
cc.datasetiHazirla(kayit_dosyasi,goruntulerin_dosya_yolu)

In [6]:
metadata_df = pd.read_csv(kayit_dosyasi)

In [7]:
metadata_df.tail()

,Id,Findings
7463,CXR998_IM-2479-1001,Cardiomediastinal silhouette demonstrates norm...
7464,CXR998_IM-2479-2001,Cardiomediastinal silhouette demonstrates norm...
7465,CXR999_IM-2480-1001,Lungs are clear bilaterally. There is no focal...
7466,CXR999_IM-2480-2001,Lungs are clear bilaterally. There is no focal...
7467,CXR999_IM-2480-3001,Lungs are clear bilaterally. There is no focal...


In [8]:
frontal_goruntulerin_dosya_yolu = './dataset/frontal_images/'
frontal_goruntulerin_raporlarinin_dosya_yolu = './dataset/frontal_reports/metadata_frontal.csv'

In [9]:
frontal_goruntulerin_dosya_isimleri = os.listdir(frontal_goruntulerin_dosya_yolu)

In [10]:
with open(frontal_goruntulerin_raporlarinin_dosya_yolu,'w',encoding='utf-8') as r: # Kayıt edilecek boş rapor csv dosyasını oluştur.
    columns = ['Id','Findings']
    csv.writer(r).writerow(columns)

In [ ]:
cc = csvCreator()
for i in range(len(metadata_df)):
    img_id = metadata_df['Id'].iloc[i][3:] +'.png'
    finding = metadata_df['Findings'].iloc[i]
    if img_id in frontal_goruntulerin_dosya_isimleri:
        cc.satirEkle(frontal_goruntulerin_raporlarinin_dosya_yolu,img_id,finding)
        print(f"{img_id} eklendi.. Finding:{finding}")